<img src="https://github.com/christopherhuntley/BUAN5405-docs/blob/master/Slides/img/Dolan.png?raw=true" width="180px" align="right">

# Lesson 9: Dictionaries
_Associative arrays by another name_

# Learning Objectives

## Theory / Be able to explain ...
- The purpose and usage of associative arrays
- Python dictionaries as associative arrays
- Hashing and it's implications for dictionary keys

## Skills / Know how to  ...
- Display the hash for any dictionary key
- Iterate over dictionary items, keys, and values 
- Generate dictionaries from keys and values
- Use a dictionary comprehension for efficient dictionary generation

**What follows is adapted from Chapter 9 of the _Python For Everybody_ book. If you have not read it, then please do so before continuing on.**

---

## The Magic of Associative Arrays
>"A very little key will open a very heavy door." -― Charles Dickens, _Hunted Down_

After so many years of programming in C, I found myself using it for basically everything, until one day in 1994 I was asked by a very wise boss to try [AWK](https://en.wikipedia.org/wiki/AWK). AWK was a text processing language developed at Bell Labs in the 1970s by the same team that created C and Unix. It was designed to be a tiny domain-specific language for working with streaming text data. One would feed data to an AWK script one line at a time. AWK would then output text to an output file, also one line at a time. It could, of course, remember things from one line to the next, allowing it to accumulate information along the way. 

Soon I was using AWK for lots of text processing tasks. One notable application was to translate mainframe data into SQL code for loading into a relational database. Data would come in one line at a time and then go right into the database. I think I got at least one promotion from just this one parlor trick. A year or so later, in late 1995 or early 1996, I used the same trick to develop a dashboarding web app that was cobbled together with AWK and bash scripts. No Perl. No Python. No PHP. No Java. Just AWK and bash on a Unix command line. I am still amazed that it worked but we never had a crash or any other bug reported.   

One of the reasons why I loved AWK so much was a feature called "associative arrays" where we could index a variable length array with text **keys** instead of integers. We could even mix keys with integer indexes if we liked. This meant that, for example, I could have an array of birthdays indexed by people's names. Or vice versa if that was what I wanted.  Or, I could create a histogram for words in a file with two lines of code. The potential uses seemed endless. Nothing could have been more convenient for a wannabe smart and lazy programmer. 

The Python equivalent of an associative array is a **dictionary**. It does many of the same things as a list but with keys instead of positions. Like associative arrays, there are an endless array of uses. If you have ever pulled data from a web API or added a Series to a DataFrame then you have used something like a dictionary. It's just how it's done. 

---
## Dictionaries as Collections of Key-Value Pairs
Python dictionaries have the type `dict`. Here's a brief example, followed by a few notes.

In [ ]:
birthdays = {'Washington':'1732-02-22','Jefferson':'1743-04-13','Lincoln':'1809-02-12'}
birthdays['Madison']='1751-03-16'
for president in birthdays:
    print(president,"was born", birthdays[president])

Washington was born 1732-02-22
Jefferson was born 1743-04-13
Lincoln was born 1809-02-12
Madison was born 1751-03-16


- `dict` literals work like `list` literals except they use curly brackets `{}` instead of square brackets `[]`.
- `dict` indexes use **keys** of any **Hashable** type (more about this in a minute) instead of just integers. 
- the bracket operator `[]` is used for retrieving specific values, just like a list. 
- Dictionaries are mutable. We can add or remove key-value pairs as needed. The `+=` operator doesn't work though.  

In [ ]:
birthdays += {'Adams Sr.':'1735-10-30'}

TypeError: ignored

In [ ]:
# practice adding a key value pair in the dict
birthdays['Adams Sr'] = '1735-10-30'
for president in birthdays:
    print(president, "was born", birthdays[president])

Washington was born 1732-02-22
Jefferson was born 1743-04-13
Lincoln was born 1809-02-12
Madison was born 1751-03-16
Adams Sr was born 1735-10-30


### Hashing
To ensure data integrity, dictionary keys are required to be:
- **Unique**: If two items have the same key, then how do we know which is which?
- **Immutable**: If we can change the value of a key (e.g., via aliasing) then how does the dictionary let everybody know about it?
- **Printable**: If not printable/visible, then how can we humans use them safely? 

When passed an object, a **hashing** function generates a _printable_ **hash** or **digest** value that is _almost_ guaranteed to be unique. The odds of "collision" (i.e., two objects with the same hash) is very, very, very remote. Further, if the object being hashed is itself immutable then we have met all three requirements for dictionary keys:

1. Each key has a unique hash. If two keys are the same then they generate the same hash.
2. Because the key is required to be immutable, then so is the hash.
3. Hashes are printable as (typically) very long strings of characters or digits. So, even if the key itself isn't printable, its hash is. 

Besides its obvious integrity advantages, hashing of keys is also highly efficient. Since hashes are convertible to strings or integers, we can sort them just like list positions. That makes using a key to lookup a value just as efficient as using an integer index to look up a value in a list. (Ever used a primary key or index to speed up a SQL query? That's exactly the same thing.)

While the precise hashing function may vary from data type to data type, the [default] uses a version of the Fowler-Noll-Vo algorithm which is outside the scope of this course. However, we can call the `hash()` standard library function on any immutable object with 100% predictable results:   

In [ ]:
print(hash( 1 ))                         # int
print(hash( 2.3 ))                       # float
print(hash( "Mary Had a Little Lamb" ))  # string
print(hash( b'Mary Had a Little Lamb' )) # bytes (same as string)
print(hash( (1,2,3) ))                   # tuple, which is immutable
print(hash( hash ))                      # the hash function object
print(hash( [1,2,3] ))                   # list; oops that's mutable!

1
691752902764107778
7509951932852457187
7509951932852457187
529344067295497451
8770448553099


TypeError: ignored

### Dictionary Traversal
When iterating over a `dict`, we can use one of three iterator _view_ methods that return list-like sequences:
- `keys()` which returns all keys
- `values()` which returns all the values
- `items()` which returns all the key-value pairs (a.k.a., "items")

When used in a `for` loop the default is to use the `keys()` iterator:

In [ ]:
# the default iteration order
for president in birthdays:
    print(president,"was born", birthdays[president])
print("---")
# explicitly iterating over keys()
for key in birthdays.keys():
    print(key,"was born", birthdays[president])

Washington was born 1732-02-22
Jefferson was born 1743-04-13
Lincoln was born 1809-02-12
Madison was born 1751-03-16
Adams Sr was born 1735-10-30
---
Washington was born 1735-10-30
Jefferson was born 1735-10-30
Lincoln was born 1735-10-30
Madison was born 1735-10-30
Adams Sr was born 1735-10-30


However, we can also iterate over items or even values, though with somewhat differing results.

In [ ]:
# iterating over items; each item is a tuple
for item in birthdays.items():
    print(item)
print("---")
# iterating over values()
for v in birthdays.values():
    print(v)

('Washington', '1732-02-22')
('Jefferson', '1743-04-13')
('Lincoln', '1809-02-12')
('Madison', '1751-03-16')
('Adams Sr', '1735-10-30')
---
1732-02-22
1743-04-13
1809-02-12
1751-03-16
1735-10-30


You may have noticed that the order is the same each time. As of Python 3.6, each iterator will always follow the order in which the keys were inserted into the dictionary. 

### **Pulse Check ...**
**Use the [`dict()` function](https://docs.python.org/3/library/stdtypes.html#dict) to create a new dictionary called `presidents` that swaps the keys and values of the `birthdays` dictionary.** Each key should be a birthdate and each value should be the associated president's last name.

In [ ]:
# YOUR CODE HERE
# c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
items = zip(birthdays.values(), birthdays.keys())
presidents = dict(items)
print(presidents)

{'1732-02-22': 'Washington', '1743-04-13': 'Jefferson', '1809-02-12': 'Lincoln', '1751-03-16': 'Madison', '1735-10-30': 'Adams Sr'}


In [ ]:
## Example from Python Dictionaries Lecture Week 6 BA505
# # reverse the 'counts' dict 
# rev_counts = {v:k for k, v in counts.items()}
# print(rev_counts)
# # get keys from the 'rev_counts' dict 
# # keep in mind that the keys are value in the original dict 
# val_lst = list(rev_counts.keys())
# # sort the value list 
# val_lst.sort()
# for val in val_lst:
#     # reason we print it out this way is that 
#     # we need to reverse it back to the original (key, value) pairs
#     print(rev_counts[val], val)


In [ ]:
presidents = {value: key for key, value in birthdays.items()}
print(presidents)

{'1732-02-22': 'Washington', '1743-04-13': 'Jefferson', '1809-02-12': 'Lincoln', '1751-03-16': 'Madison', '1735-10-30': 'Adams Sr'}


In [ ]:
# Huntley's solution using a list comprehension
items = [[b[1], b[0]] for b in birthdays.items()]
presidents = dict(items)
print(presidents)


{'1732-02-22': 'Washington', '1743-04-13': 'Jefferson', '1809-02-12': 'Lincoln', '1751-03-16': 'Madison', '1735-10-30': 'Adams Sr'}


In [ ]:
#@title <--- Check your work
%%html
<div style="max-width: 1000px">
   <div style="position: relative;padding-bottom: 56.25%;height: 0;">
     <iframe style="position: absolute;top: 0;left: 0;width: 100%;height: 100%;" rel="0" modestbranding="1"  
     src="https://www.youtube.com/embed/DeJXfkTXFnk"
     frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
   </div>
</div>

---
## Pro Tips

### Generating `dict`s
In the examples so far, all of our `dict`s have been created as literals with `{}` or through the `dict()` function. However, dictionaries can be created in lots of curious ways. Just about any iteration process that generates paired sequences of keys and values can be used to create and populate a dictionary. 

In [ ]:
d_keys =   ["Washington","Jefferson"]
d_values = ['1732-02-22','1743-04-13']

d = {}        # an empty dictionary
for i in range(len(d_keys)):
    d[d_keys[i]] = d_values[i]
d

{'Washington': '1732-02-22', 'Jefferson': '1743-04-13'}

While straightforward, this is not the most efficient way to generate a dictionary. There are actually two different one line equivalents that are both less code and more efficient. Both are explained below.

### `dict` Comprehensions
A dictionary comprehension is a lot like a list comprehension, which we covered in Lesson 8:
```python
{ key : value for item in items }
```
The key and/or value will vary from item to item.

In [ ]:
# reuses the d_keys and d_values from before

{ d_keys[i] : d_values[i] for i in range(len(d_keys)) }

{'Washington': '1732-02-22', 'Jefferson': '1743-04-13'}

There are other allowed forms (e.g., the pairs can be specified as tuples) but this is the most commonly used one. 

### That One Weird Zip Dict Trick (Say that fast 3 times)
The `zip()` function converts several sequences of the same length into an iterator of tuples (immutable lists, covered in Lesson 10), where the each tuple is composed of corresponding items. 

In [ ]:
bdays = ['1732-02-22','1743-04-13','1809-02-12']
presidents = ['Washington','Jefferson','Lincoln']

z = zip(bdays,presidents)  # z is an iterator
list(z)                    

[('1732-02-22', 'Washington'),
 ('1743-04-13', 'Jefferson'),
 ('1809-02-12', 'Lincoln')]

This can be very useful for generating dictionaries. Let one of the sequences be a list of keys and the other a list of values. When used with the `dict()` constructor we now have a quick and efficient way to zip the keys and values together into a single dict.

In [ ]:
# bdays is the keys list
# presidents is the values list
dict(zip(bdays,presidents))  # Voila! a one line dict maker

{'1732-02-22': 'Washington',
 '1743-04-13': 'Jefferson',
 '1809-02-12': 'Lincoln'}

---
## **Exercises**

**1. Use your `waist2Hip_ratio()` function to process each of the dictionaries listed below.**
```python
[{'waist': 28, 'hip': 40, 'gender': 'F'},
 {'waist': 23, 'hip': 35, 'gender': 'F'},
 {'waist': 30, 'hip': 40, 'gender': 'M'},
 {'waist': 30, 'hip': 37, 'gender': 'M'},
 {'waist': 32, 'hip': 39, 'gender': 'M'}]
```

In [ ]:
# creating a triplet of lists within list in function call
# w2h_ratio from lesson 6/ revised in module 2b workshop
# applied lists in lesson 8 to triplet of waist, hip, gender combinations
# Exercise below from Module2b workshop
def w2h_ratio(waist_inches, hip_inches, gender):
    try:
        waist = float(waist_inches)
        hip = float(hip_inches)
    
    except:
        return "w2h_ratio: Invalid measurement(s)"
    if waist <= 0 or hip <= 0:
        return "w2h_ratio: Invalid measurement(s)"
    if gender != "M" and gender != "F":
        return "w2h_ratio: Unknown gender"
    
    waist_2_hip = waist/hip

    if gender == "F" and waist_2_hip < 0.8:
        return f"For a {gender} with waist {waist} and hip {hip} the w2h ratio is {waist_2_hip:.2f} with shape Pear."
    elif gender == "F" and waist_2_hip >= 0.8:
        return f"For a {gender} with waist {waist} and hip {hip} the w2h ratio is {waist_2_hip:.2f} with shape Apple."
    elif gender == "M" and waist_2_hip < 0.9:
        return f"For a {gender} with waist {waist} and hip {hip} the w2h ratio is {waist_2_hip:.2f} with shape Pear."
    elif gender == "M" and waist_2_hip >= 0.9:
        return f"For a {gender} with waist {waist} and hip {hip} the w2h ratio is {waist_2_hip:.2f} with shape Apple."

##UPDATE HERE 
combinations = [[28,40,'F'],[23, 35, 'F'],[30,40,'M'],[30,37,'M'],[32,39,'M']] # a list of lists containing the waist, hip, and gender combinations

# iterate through 'combinations' list 
for combo in combinations:
    print(w2h_ratio(combo[0],combo[1],combo[2]))  # print out first index (0) as waist, second index(1) as hip, and third index (2) as gender

For a F with waist 28.0 and hip 40.0 the w2h ratio is 0.70 with shape Pear.
For a F with waist 23.0 and hip 35.0 the w2h ratio is 0.66 with shape Pear.
For a M with waist 30.0 and hip 40.0 the w2h ratio is 0.75 with shape Pear.
For a M with waist 30.0 and hip 37.0 the w2h ratio is 0.81 with shape Pear.
For a M with waist 32.0 and hip 39.0 the w2h ratio is 0.82 with shape Pear.


In [ ]:
# creating a triplet of lists within list in function call
# w2h_ratio from lesson 6/ revised in module 2b workshop
# applied lists in lesson 8 to triplet of waist, hip, gender combinations
# Exercise below from Module2b workshop
def w2h_ratio(waist_inches, hip_inches, gender):
    try:
        waist = float(waist_inches)
        hip = float(hip_inches)
    
    except:
        return "w2h_ratio: Invalid measurement(s)"
    if waist <= 0 or hip <= 0:
        return "w2h_ratio: Invalid measurement(s)"
    if gender != "M" and gender != "F":
        return "w2h_ratio: Unknown gender"
    
    waist_2_hip = waist/hip

    if gender == "F" and waist_2_hip < 0.8:
        return f"For a {gender} with waist {waist} and hip {hip} the w2h ratio is {waist_2_hip:.2f} with shape Pear."
    elif gender == "F" and waist_2_hip >= 0.8:
        return f"For a {gender} with waist {waist} and hip {hip} the w2h ratio is {waist_2_hip:.2f} with shape Apple."
    elif gender == "M" and waist_2_hip < 0.9:
        return f"For a {gender} with waist {waist} and hip {hip} the w2h ratio is {waist_2_hip:.2f} with shape Pear."
    elif gender == "M" and waist_2_hip >= 0.9:
        return f"For a {gender} with waist {waist} and hip {hip} the w2h ratio is {waist_2_hip:.2f} with shape Apple."

##UPDATE HERE 
# list of dictionaries for each measurement waist, hip, and gender
list_of_dict = [{'waist': 28, 'hip': 40, 'gender': 'F'},{'waist': 23, 'hip': 35, 'gender': 'F'},{'waist': 30, 'hip': 40, 'gender': 'M'},{'waist': 30, 'hip': 37, 'gender': 'M'},{'waist': 32, 'hip': 39, 'gender': 'M'}]

## Return the following dicts in the 
for combo in list_of_dict:
     print(w2h_ratio(combo['waist'], combo['hip'], combo['gender'])) # each function input(measurement) in the value associated with the keys 'waist','hip','gender'

For a F with waist 28.0 and hip 40.0 the w2h ratio is 0.70 with shape Pear.
For a F with waist 23.0 and hip 35.0 the w2h ratio is 0.66 with shape Pear.
For a M with waist 30.0 and hip 40.0 the w2h ratio is 0.75 with shape Pear.
For a M with waist 30.0 and hip 37.0 the w2h ratio is 0.81 with shape Pear.
For a M with waist 32.0 and hip 39.0 the w2h ratio is 0.82 with shape Pear.


In [ ]:
#@title <--- Check your work
%%html
<div style="max-width: 1000px">
   <div style="position: relative;padding-bottom: 56.25%;height: 0;">
     <iframe style="position: absolute;top: 0;left: 0;width: 100%;height: 100%;" rel="0" modestbranding="1"  
     src="https://www.youtube.com/embed/1Cz4eS2-wRA"
     frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
   </div>
</div>

**2. Rewrite your `roman2int()` function from Lesson 6 to use a dictionary and an iterator instead of the 14-clause `if` statement. The dictionary should have 13 keys, one for each of the patterns.** 

In [ ]:
# From workshop 3a/lesson 6 
def roman2int(roman_numeral): #positional argument is a string of roman numeral characters
    int_value  = 0 #initialize accumulator total numerical sum 
    remaining = roman_numeral # counter variable is the character in the roman numerical  
    while remaining: # if characters remain in the roman_numeral
        if remaining[0:2] == "IV": # if the first two characters of roman_numeral are "IV"
            int_value += 4         # update the accumulator total numerical sum of 'int_value'
            remaining = remaining[2:]  # update the remaining characters of 'roman_numeral' slicing off the first two characters

        elif remaining[0:2] == "IX":
            int_value += 9 
            remaining = remaining[2:]
        
        elif remaining[0:2] == "XL":
            int_value += 40 
            remaining = remaining[2:]
        
        elif remaining[0:2] == "XC":
            int_value += 90 
            remaining = remaining[2:]
        
        elif remaining[0:2] == "CD":
            int_value += 400 
            remaining = remaining[2:] 
        
        elif remaining[0:2] == "CM":
            int_value += 900
            remaining = remaining[2:]
          
        elif remaining[0] == "I":  # if first character of roman_numeral is "I"
            int_value += 1           # update the numerical sum of 'int_value'
            remaining = remaining[1:] # update the remaining characters of 'roman_numeral' slicing off the first character
        
        elif remaining[0] == "V":
            int_value += 5
            remaining = remaining[1:]
         
        elif remaining[0] == "X":
            int_value += 10
            remaining = remaining[1:]

        elif remaining[0] == "L":
            int_value += 50
            remaining = remaining[1:]
        
        elif remaining[0] == "C":
            int_value += 100
            remaining = remaining[1:]

        elif remaining[0] == "D":
            int_value += 500
            remaining = remaining[1:]

        elif remaining[0] == "M":
            int_value += 1000
            remaining = remaining[1:]

        else:
            remaining = remaining[1:]

    return int_value    # after all characters of roman_numeral have been exhausted return the total numerical sum of 'roman_numeral'


# Function call testing "XXIX"
roman2int("XXIX")
          

29

In [ ]:
def roman2int(roman_numeral): #positional argument is a string of roman numeral characters
    int_value  = 0 #initialize accumulator total numerical sum 
    # key-value pairs for roman_numeral character and its int_value
    roman_dict = {"IV" : 4, "IX" : 9, "XL" : 40, "XC": 90, "CD" : 400, "CM" : 900, "I" : 1, "V" : 5, "X" : 10, "L" : 50, "C" : 100, "D" : 500, "M": 1000}
 
    for c in roman_numeral:
        if c in roman_dict.keys():
           int_value += roman_dict[c]
             
    return int_value

roman2int("XXIX") # gives us the incorrect sum we need to account for the first or first two characters

31

In [ ]:
## change to a while loop 

def roman2int(roman_numeral):
    ''' 
    This function returns the total numerical sum of a given roman_numeral string
    '''
    roman_dict = {"IV" : 4, "IX" : 9, "XL" : 40, "XC": 90, "CD" : 400, "CM" : 900, "I" : 1, "V" : 5, "X" : 10, "L" : 50, "C" : 100, "D" : 500, "M": 1000}
    
    remaining = str(roman_numeral) # to ensure our roman numeral is a string data type
    int_value = 0 # initialize our accumulator which represents the total sum/numerical value of the roman numeral

    while remaining:
    # Check the first two characters of the roman_numeral are in our roman_dict
        if remaining[:2] in roman_dict.keys(): # if the first two characters of the roman numeral starts with any of the dict keys
            int_value += roman_dict[remaining[:2]] # total sum of roman_numeral is updated by the value of the first two characters(retrieved by its key)
            remaining = remaining[2:] # update our roman_numeral to contain the preceeding characters

        # check if the proceeding single characters of the roman_numeral are in our roman_dict
        elif remaining[0] in roman_dict.keys(): # if the first character of the roman numeral starts with any of the keys of the dict
            int_value += roman_dict[remaining[0]] # total sum of roman_numeral is updated by the value of the first character(retrieved by its key)
            remaining = remaining[1:] # update our roman_numeral to contain the preceeding characters

        else:
            remaining = remaining[1:] # Otherwise update our roman_numeral to containg the preceeding characters

    return int_value  # return our total sum of the roman_numeral

print(roman2int("XXIX"))

       

29


---
## **Before you go ... Submit your work on Google Classroom**
- Save your notebook to be sure it is up to date.
- In Google Drive, drag it into the `BUAN5405` folder so that you can find it again later.
- Go to the assignment in Google Classroom. 
- Turn in your notebook. Your notebook will become read-only. 
- Once it has been reviewed it will be returned and no-longer be read-only.

---
> ## Every Tee Shirt Has a Story
> ABOUT THE SLASHDOT EFFECT     
> I discovered Slashdot my first semester at Fairfield in 1997. I was trying to figure out how to explain geek culture to my undergrad business students. "You mean you actually call yourselves geeks? ..." Why yes we did, and there was even this new website called Slashdot that billed itself as "News for Nerds" that was catching on with programmers around the world. Even the name was total geek, a play on how web newbies would read URLs aloud. The full URL was `http://slashot.org`. Try saying that out loud, including punctuation, a couple times.  
> 
> In keeping with its motto, it was a sort of curated blog with open discussion (without a log in required) that linked to the freshest nerd content from around the web. Within a few months of its founding, it had a continuous, 24x7 readership in the millions, all very geeky and loyal. This was at a time when getting a few thousand people to visit a major media site was a big deal. 
> 
> Slashdot became famous with the non-geek crowd when the blog postings began linking to not-so-nerdy newspapers, magazines, political candidates, and other media providers that were not prepared for **2 million** geeks clicking the link at _exactly_ the same time. Servers would fry and then seize up for good, leaving people wondering what the heck just happened and asking about backups. Such was the [Slashdot Effect](https://en.wikipedia.org/wiki/Slashdot_effect).  
>
> I picked up this tee at a Linux World show circa 2006. By then Slashdot had faded a bit, though was still popular in the right circles. The site is still around, though with a much, much smaller readership. It lost a lot of the original geek readership when Condé Nast bought it around the time I picked up the tee shirt.   


![L9 Tee Front](https://github.com/christopherhuntley/BUAN5405-docs/raw/master/Photos/L09_TeeFront.jpeg)

## Copyright &copy; 2020 Christopher Huntley. All rights reserved. 